In [1]:
cd drive/Othercomputers/내 노트북/googleDrive/Hackathon/

/content/drive/Othercomputers/내 노트북/googleDrive/Hackathon


In [32]:
pip --version

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)


In [2]:
!pip install pandas tqdm
!pip install sentencepiece
!pip install transformers==3
!pip install torch

     |████████████████████████████████| 1.2 MB 8.4 MB/s 
     |████████████████████████████████| 754 kB 7.1 MB/s 
     |████████████████████████████████| 895 kB 50.5 MB/s 
     |████████████████████████████████| 3.0 MB 45.3 MB/s 


In [3]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-l9qpk15e
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-l9qpk15e
  Created wheel for kobert: filename=kobert-0.1.2-py3-none-any.whl size=12771 sha256=a605c3656f316ead9a46e2297013e11bdc2ca8b960df188783a7f3343fed54cc
  Stored in directory: /tmp/pip-ephem-wheel-cache-pdj8m33r/wheels/d3/68/ca/334747dfb038313b49cf71f84832a33372f3470d9ddfd051c0
Successfully built kobert


In [7]:
!pip install gluonnlp

     |████████████████████████████████| 344 kB 9.9 MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp37-cp37m-linux_x86_64.whl size=595732 sha256=af237c005c54ac3efa0161fcd6e0e5bc6638fe94d5339aaae8bfc87ae9afce30
  Stored in directory: /root/.cache/pip/wheels/be/b4/06/7f3fdfaf707e6b5e98b79c041e023acffbe395d78a527eae00
Successfully built gluonnlp


In [9]:
!pip install mxnet

     |████████████████████████████████| 46.9 MB 2.2 MB/s 
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [10]:
import torch
import torch.nn as nn
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert.pytorch_kobert import get_pytorch_kobert_model

In [27]:
max_len = 100
batch_size = 64
warmup_ratio = 0.1
num_epochs = 4
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [25]:
import numpy as np

In [11]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=2,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [12]:
device = torch.device("cpu")

In [13]:
bertmodel, vocab = get_pytorch_kobert_model()
model = BERTClassifier(bertmodel,  dr_rate=0.5)
model.load_state_dict(torch.load('./model.pt', map_location=device))

[██████████████████████████████████████████████████]
[██████████████████████████████████████████████████]


<All keys matched successfully>

In [20]:
from torch.utils.data import Dataset, DataLoader

In [16]:
#토큰화
from kobert.utils import get_tokenizer

import gluonnlp as nlp
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=5)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append(False)
            elif np.argmax(logits) == 1:
                test_eval.append(True)

        print(test_eval[0])

using cached model


In [21]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]
        #self.labels = [j for i, j in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [28]:
end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 시발


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


True


하고싶은 말을 입력해주세요 : 병신
True


하고싶은 말을 입력해주세요 : 차경민
False


하고싶은 말을 입력해주세요 : 한남충들 개극혐 우웩
True




KeyboardInterrupt: ignored

In [30]:
pip --version

pip 21.1.3 from /usr/local/lib/python3.7/dist-packages/pip (python 3.7)
